# QuTiP example: Landau-Zener-Stuckelberg inteferometry

J.R. Johansson and P.D. Nation

For more information about QuTiP see [http://qutip.org](http://qutip.org)

In [85]:
%matplotlib inline

In [86]:
import matplotlib.pyplot as plt

In [87]:
import numpy as np

In [88]:
from qutip import *
from qutip.ui.progressbar import TextProgressBar as ProgressBar

Landau-Zener-Stuckelberg interferometry: Steady state of a strongly driven two-level system, using the one-period propagator. 

In [89]:
# set up the parameters and start calculation
wq  = 0.8  * 2 * np.pi  # qubit sigma_z coefficient
wqa = 1.8095573684677206
wc  = 1.0  * 2 * np.pi  # cavity freq
g   = 0.6  * 2 * np.pi  # coupling strength
ga  = 0.02 * 2 * np.pi
w   = 2.0  * 2 * np.pi  # driving frequency
T   = 2.0  / w * np.pi      # driving period 
N   = 8

gamma1 = 0.00001        # relaxation rate
gamma2 = 0.0001          # dephasing  rate
gamma3 = 0.0001

eps_list = np.linspace(-0.5, 0.5, 31) * 2.0 * np.pi  # offset
A_list   = np.linspace( 0.0, 0.5, 31) * 2.0 * np.pi  # amp

# pre-calculate the necessary operators
sx = sigmax()
sz = sigmaz()

nq  = tensor(qeye(N), sz, qeye(2))
nqa = tensor(qeye(N), qeye(2), sz)
nc  = tensor(destroy(N).dag() * destroy(N), qeye(2), qeye(2))
sn  = tensor(qeye(N), num(2), qeye(2))
sna = tensor(qeye(N), qeye(2), num(2))

a   = tensor(destroy(N), qeye(2), qeye(2)) # destroy of cavity
sm  = tensor(qeye(N), destroy(2), qeye(2)) # destroy of qubit
sam = tensor(qeye(N), qeye(2), destroy(2)) # destroy of ancilla

# collapse operators
c_op_list = [np.sqrt(gamma1) * a, np.sqrt(gamma2) * nq, np.sqrt(gamma3) * nqa]  # relaxation and dephasing

In [90]:
# ODE settings (for list-str format)
options = Options()
options.rhs_reuse = True
options.atol = 1e-6 # reduce accuracy to speed
options.rtol = 1e-5 # up the calculation a bit

In [91]:
# for function-callback style time-dependence
def hamiltonian_t(t, args):
    """ evaluate the hamiltonian at time t. """
    H0 = args[0]
    H1 = args[1]
    w  = args[2]
    return H0 + HA * np.sin(w * t)

In [92]:
# perform the calculation for each combination of eps and A, store the result
# in a matrix
def calculate():

    pc_mat = np.zeros((len(eps_list), len(A_list)))
    pa_mat = np.zeros((len(eps_list), len(A_list)))
    pq_mat = np.zeros((len(eps_list), len(A_list)))

    pbar = ProgressBar(len(eps_list))
    
    for m, eps in enumerate(eps_list):
        H0 = 0.5 * eps * nqa + 0.5 * wq *nq + wc * nc + g * (sm + sm.dag()) * (a + a.dag()) + ga * (sm + sm.dag) * (a + a.dag())

        pbar.update(m)

        for n, A in enumerate(A_list):
            H1 = (A/2) * nqa

            # function callback format
            # args = (H0, H1, w); H_td = hamiltonian_t

            # list-str format
            #args = {'w': w}; H_td = [H0, [H1, 'sin(w * t)']]

            # list-function format
            args = w; H_td = [H0, [H1, lambda t, w: np.sin(w * t)]]

            U = propagator(H_td, T, c_op_list, args, options)
            rho_ss = propagator_steadystate(U)
            rho_a_ss = rho_ss.ptrace(2)
            rho_q_ss = rho_ss.ptrace(1)
            rho_c_ss = rho_ss.ptrace(0)
            
            pc_mat[m, n] = np.real(expect(destroy(N).dag() * destroy(N), rho_c_ss))
            pa_mat[m, n] = np.real(expect(num(2), rho_a_ss))
            pq_mat[m, n] = np.real(expect(num(2), rho_q_ss))
            print(m, n)
    return pc_mat, pa_mat, pq_mat

In [ ]:
pc_mat, pa_mat, pq_mat = calculate()

/usr/local/lib/python3.5/dist-packages/qutip/qobj.py:305: UserWarning: Initializing Qobj from unsupported type: <class 'method'>
  builtins.type(inpt))


0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

A_mat, eps_mat = np.meshgrid(A_list/(2*np.pi), eps_list/(2*np.pi))

ax.pcolor(eps_mat, A_mat, pq_mat)
ax.set_xlabel(r'Bias point $\epsilon$')
ax.set_ylabel(r'Amplitude $A$')
ax.set_title("Steadystate excitation probability\n" +
             r'$H = -\frac{1}{2}\Delta\sigma_x -\frac{1}{2}\epsilon\sigma_z - \frac{1}{2}A\sin(\omega t)$' + "\n");

In [ ]:
fig, bx = plt.subplots(figsize=(8, 8))

A_mat, eps_mat = np.meshgrid(A_list/(2*np.pi), eps_list/(2*np.pi))

bx.pcolor(eps_mat, A_mat, pa_mat)
bx.set_xlabel(r'Bias point $\epsilon$')
bx.set_ylabel(r'Amplitude $A$')
bx.set_title("Steadystate excitation probability\n" +
             r'$H = -\frac{1}{2}\Delta\sigma_x -\frac{1}{2}\epsilon\sigma_z - \frac{1}{2}A\sin(\omega t)$' + "\n");

In [ ]:
fig, cx = plt.subplots(figsize=(8, 8))

A_mat, eps_mat = np.meshgrid(A_list/(2*np.pi), eps_list/(2*np.pi))

cx.pcolor(eps_mat, A_mat, pc_mat)
cx.set_xlabel(r'Bias point $\epsilon$')
cx.set_ylabel(r'Amplitude $A$')
cx.set_title("Steadystate excitation probability\n" +
             r'$H = -\frac{1}{2}\Delta\sigma_x -\frac{1}{2}\epsilon\sigma_z - \frac{1}{2}A\sin(\omega t)$' + "\n");

In [ ]:
coherent(N, np.sqrt(4))

## Versions

In [ ]:
from qutip.ipynbtools import version_table

version_table()